In [1]:
from typing import *
from cps import *
from bt_search import *
from puzzleParser import *
from puzzleParser import analyze_puzzle_text


In [2]:
import pandas as pd
zebraLogicBench = pd.read_parquet("hf://datasets/allenai/ZebraLogicBench/grid_mode/test-00000-of-00001.parquet")

In [3]:
zebraLogicBench[0:2]

,id,size,puzzle,solution,created_at
0,lgp-test-5x6-16,5*6,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Nationality', 'B...",2024-07-11T01:22:10.734298
1,lgp-test-4x4-27,4*4,"There are 4 houses, numbered 1 to 4 from left ...","{'header': ['House', 'Name', 'Occupation', 'Bo...",2024-07-11T01:22:10.732670


In [4]:
puzzle = zebraLogicBench['puzzle'][1]
p1 = analyze_puzzle_text(puzzle)

print(p1)

Houses: 4
Vars:
 [Alice, Eric, Arnold, Peter] Each person has a unique name
 [artist, engineer, teacher, doctor] Each person has an occupation
 [fantasy, science fiction, mystery, romance] People have unique favorite book genres
 [google pixel 6, iphone 13, oneplus 9, samsung galaxy s21] People use unique phone models
Clues:
 ['engineer', 'samsung galaxy s21'] -> dLeftOf; The person who is an engineer is directly left of the person who uses a Samsung Galaxy S21
 ['fantasy'] -> is2; The person who loves fantasy books is in the second house
 ['Alice'] -> not2; Alice is not in the second house
 ['Eric', 'teacher'] -> equal; Eric is the person who is a teacher
 ['samsung galaxy s21', 'fantasy'] -> equal; The person who uses a Samsung Galaxy S21 is the person who loves fantasy books
 ['iphone 13', 'science fiction'] -> equal; The person who uses an iPhone 13 is the person who loves science fiction books
 ['science fiction', 'oneplus 9'] -> leftOf; The person who loves science fiction books 

In [5]:
def next_door(a, b):
    return a == b - 1 or a == b + 1

def right_next_to(a, b):
    return (b + 1) == a 

def direct_left_of(a, b):
    # return (a - 1) == b
    return (a + 1) == b

def left_of(a, b):
    return a < b

def right_of(a, b):
    return a > b

def one_between(a, b):
    return (a + 2) == b or (a - 2) == b

def two_between(a, b):
    return (a + 3) == b or (a - 3) == b


def configure_cps(puzzle: PzPuzzleDefinition):
    if not puzzle.is_valid():
        raise Exception("Can not generate cps for invalid puzzle definition")
    
    variables = []
    for g in puzzle.variables:
        for v in g.variables:
            variables.append(v)
    
    values = list(range(1, puzzle.house_count + 1))
    
    config = CpsConfiguration[str, int](variables, values)
    
    for g in puzzle.variables:
        config.allNotEqual(g.variables)
    
    for c in puzzle.clues:
        if c.function == "equal":
            config.equal(c.variables[0], c.variables[1])
        elif c.function == "dLeftOf":
            config.addConstraintRev(c.variables[0], c.variables[1], direct_left_of)
        elif c.function == "leftOf":
            config.addConstraintRev(c.variables[0], c.variables[1], left_of)
        elif c.function == "rightOf":
            config.addConstraintRev(c.variables[0], c.variables[1], right_of)
        elif c.function == "nextTo":
            config.addConstraintRev(c.variables[0], c.variables[1], next_door)
        elif c.function == "oneBetween":
            config.addConstraintRev(c.variables[0], c.variables[1], one_between)
        elif c.function == "twoBetween":
            config.addConstraintRev(c.variables[0], c.variables[1], two_between)


        elif c.function == "not1":
            config.mustNotBe(c.variables[0], 1)    
        elif c.function == "not2":
            config.mustNotBe(c.variables[0], 2)
        elif c.function == "not3":
            config.mustNotBe(c.variables[0], 3)    
        elif c.function == "not4":
            config.mustNotBe(c.variables[0], 4)
        elif c.function == "not5":
            config.mustNotBe(c.variables[0], 5)    
        elif c.function == "not6":
            config.mustNotBe(c.variables[0], 6)       
             
        elif c.function == "is1":
            config.mustBe(c.variables[0], 1)    
        elif c.function == "is2":
            config.mustBe(c.variables[0], 2)
        elif c.function == "is3":
            config.mustBe(c.variables[0], 3)    
        elif c.function == "is4":
            config.mustBe(c.variables[0], 4)
        elif c.function == "is5":
            config.mustBe(c.variables[0], 5)    
        elif c.function == "is6":
            config.mustBe(c.variables[0], 6)
        else:
            raise Exception(f'Function not implemented: "{c.function}"')
    
    return config




In [6]:
puzzle = zebraLogicBench['puzzle'][41]
p1 = analyze_puzzle_text(puzzle)
cpsConf = configure_cps(p1)
display(cpsConf)
baseState = CpsState(cpsConf)
# result = BtSearch.search(SimpleBtSearch(), baseState)
result = BtSearch.search(MrvBtSearch(), baseState)

print(puzzle)
display(p1)

get_bt_result(result)

CPS-Config {
 Values: [
  1
  2
  3
 ]
 Variables: [
  Peter
  Eric
  Arnold
  red
  white
  yellow
  child is named Fred
  mother of Fred
  child is named Meredith
  mother of Meredith
  child is named Bella
  mother of Bella
 ]
Constraints: [
(Peter, Eric) => lambda a, b: a != b
(Peter, Arnold) => lambda a, b: a != b
(Eric, Peter) => lambda a, b: a != b
(Eric, Arnold) => lambda a, b: a != b
(Eric, child is named Fred) => REV: def left_of(a, b):
    return a < b

(Arnold, Peter) => lambda a, b: a != b
(Arnold, Eric) => lambda a, b: a != b
(Arnold, red) => lambda a, b: a == b
(red, white) => lambda a, b: a != b
(red, yellow) => lambda a, b: a != b
(red, Arnold) => REV: lambda a, b: a == b
(red, None) => lambda a, b: predicate(a)
(white, red) => lambda a, b: a != b
(white, yellow) => lambda a, b: a != b
(white, child is named Meredith) => lambda a, b: a == b
(yellow, red) => lambda a, b: a != b
(yellow, white) => lambda a, b: a != b
(child is named Fred, mother of Fred) => lambda a, b: 

There are 3 houses, numbered 1 to 3 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Peter`, `Eric`, `Arnold`
 - Each person has a favorite color: `red`, `white`, `yellow`
 - Each mother is accompanied by their child: `Fred`, `Meredith`, `Bella`

## Clues:
1. Arnold is the person whose favorite color is red.
2. The person's child is named Fred is somewhere to the left of Eric.
3. The person whose favorite color is red is in the second house.
4. The person's child is named Bella is in the first house.
5. The person who loves white is the person's child is named Meredith.



Houses: 3
Vars:
 [Peter, Eric, Arnold] Each person has a unique name
 [red, white, yellow] Each person has a favorite color
 [child is named Fred, mother of Fred, child is named Meredith, mother of Meredith, child is named Bella, mother of Bella] Each mother is accompanied by their child
Clues:
 ['Arnold', 'red'] -> equal; Arnold is the person whose favorite color is red
 ['child is named Fred', 'Eric'] -> leftOf; The person's child is named Fred is somewhere to the left of Eric
 ['red'] -> is2; The person whose favorite color is red is in the second house
 ['child is named Bella'] -> is1; The person's child is named Bella is in the first house
 ['white', 'child is named Meredith'] -> equal; The person who loves white is the person's child is named Meredith

Step count:  5
No result found


{}

In [7]:
failed_count = 0
failed = []

## PUZZLE 176 => max recurse error
puzzle_count = 150

index = 0
for puzzle in zebraLogicBench['puzzle'][0:puzzle_count]:
    
    p1 = analyze_puzzle_text(puzzle)
    cpsConf = configure_cps(p1)
    baseState = CpsState(cpsConf)
    # result = BtSearch.search(SimpleBtSearch(), baseState)
    result = BtSearch.search(MrvBtSearch(), baseState)
    
    if result.result is None:
        failed.append(index)
        failed_count += 1
    
    index += 1
    
display(failed_count)
display(failed)
display(failed_count / puzzle_count)

16

[2, 3, 27, 32, 41, 43, 45, 51, 56, 65, 77, 80, 85, 102, 108, 138]

0.10666666666666667